In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/content/heart.csv')
print("🔹 Dataset Loaded Successfully!")
data.head(10)

In [ ]:
data.info()

In [ ]:
data.describe().transpose()

In [ ]:
data.columns

In [ ]:
num_cols = []
for column in data.columns:
  if ((data[column].dtype != 'object') & (data[column].nunique() > 2)):
    num_cols.append(column)
print(num_cols)

In [ ]:
for column in num_cols:
  sns.boxplot(data=data[column], orient='h')
  plt.xlabel(column)
  plt.show()

In [ ]:
for column in num_cols:
  q3, q1 = np.percentile(data[column], [75, 25])
  IQR = q3 - q1
  max_boxPlot = q3 + (1.5 * IQR)
  min_boxPlot = q1 - (1.5 * IQR)
  data = data[(data[column] > min_boxPlot) & (data[column] < max_boxPlot)]

In [ ]:
data.shape

In [ ]:
for column in num_cols:
  sns.boxplot(data=data[column], orient='h')
  plt.xlabel(column)
  plt.show()

In [ ]:
data.info()

In [ ]:
(data.isnull().sum())

In [ ]:
cat_cols = data.select_dtypes(include='object').columns
cat_cols

In [ ]:
for column in cat_cols:
  sns.countplot(data=data, x=column, hue="HeartDisease")
  plt.show()

In [ ]:
sns.countplot(data=data, x="FastingBS", hue="HeartDisease")
plt.show()

In [ ]:
data['HeartDisease'].value_counts()

In [ ]:
encoded_data = pd.get_dummies(data, drop_first=True)
# Convert boolean columns to integers (0/1)
encoded_data = encoded_data.astype(int)
print("\n🎨 One-Hot Encoding Applied!")
print(encoded_data.head())

In [ ]:
encoded_data.head(10)

In [ ]:
data.shape

In [ ]:
# Split into features and target
x = encoded_data.drop('HeartDisease', axis=1)
y = encoded_data['HeartDisease']


In [ ]:
# Train/test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Scale the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)


In [ ]:
# Convert back to DataFrame for readability (optional)
x_train_scaled_df = pd.DataFrame(x_train_scaled, columns=x_train.columns)
x_test_scaled_df = pd.DataFrame(x_test_scaled, columns=x_test.columns)

In [ ]:
# ANN Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(16, input_shape=(x_train.shape[1],), activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_scaled, y_train, validation_split=0.2, epochs=50, batch_size=16)

# Evaluate
loss, accuracy = model.evaluate(x_test_scaled, y_test)
print(f"Test Loss: {loss:.4f}, Accuracy: {accuracy * 100:.2f}%")# Plot training history
plt.figure(figsize=(12, 5))

# Accuracy curve
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Val Accuracy', marker='o')
plt.title('📈 Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss curve
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss', marker='o')
plt.plot(history.history['val_loss'], label='Val Loss', marker='o')
plt.title('📉 Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Predict on test set
y_pred_prob = model.predict(x_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

# Confusion Matrix and Classification Report
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

# Display Confusion Matrix
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Disease', 'Disease'], yticklabels=['No Disease', 'Disease'])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("🧾 Confusion Matrix")
plt.show()

# Print Classification Report
print("📊 Classification Report:\n", cr)


In [ ]:
# Save model and scaler
model.save('/content/heart_failure_prediction_model.h5')
import joblib
joblib.dump(scaler, '/content/heart_scaler.save')

In [ ]:
import pandas as pd
import tensorflow as tf
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load saved model and scaler
model = tf.keras.models.load_model('/content/heart_failure_prediction_model.h5')
scaler = joblib.load('/content/heart_scaler.save')

# Get feature list
feature_columns = pd.read_csv('/content/heart.csv')
encoded_columns = pd.get_dummies(feature_columns, drop_first=True).astype(int)
x_columns = encoded_columns.drop('HeartDisease', axis=1).columns.tolist()

# In-memory users DB
users_db = {}

def login_screen():
    clear_output()
    print("🔐 Heart Failure Predictor - Login")

    username = widgets.Text(description="Username:")
    password = widgets.Password(description="Password:")
    login_btn = widgets.Button(description="Login", button_style='success')
    register_btn = widgets.Button(description="Register", button_style='info')

    def do_login(b):
        if users_db.get(username.value) == password.value:
            print("✅ Login successful!")
            input_form_ui()
        else:
            print("❌ Invalid credentials.")

    def do_register(b):
        if username.value in users_db:
            print("⚠️ Username already exists.")
        else:
            users_db[username.value] = password.value
            print("✅ Registered. Please login.")

    login_btn.on_click(do_login)
    register_btn.on_click(do_register)

    display(username, password, login_btn, register_btn)

def input_form_ui():
    clear_output()
    print("📥 Enter Patient Data for Prediction")

    # Mandatory inputs
    age = widgets.IntText(description="Age*", value=50)
    bp = widgets.IntText(description="RestingBP*", value=120)
    chol = widgets.IntText(description="Cholesterol*", value=200)
    fbs = widgets.Dropdown(options=[0, 1], description="FastingBS*")
    hr = widgets.IntText(description="MaxHR*", value=150)
    oldpeak = widgets.FloatText(description="Oldpeak*", value=1.0)

    # Optional dummy variables
    sex_m = widgets.Dropdown(description="Sex_M", options=[0, 1])
    cp_na = widgets.Dropdown(description="CP_Non-Anginal", options=[0, 1])
    ecg_normal = widgets.Dropdown(description="ECG_Normal", options=[0, 1])
    ex_angina = widgets.Dropdown(description="Exercise_Y", options=[0, 1])
    slope_up = widgets.Dropdown(description="ST_Slope_Up", options=[0, 1])
    slope_flat = widgets.Dropdown(description="ST_Slope_Flat", options=[0, 1])

    predict_btn = widgets.Button(description="🔍 Predict", button_style='primary')

    def make_prediction(b):
        input_dict = {
            'Age': age.value, 'RestingBP': bp.value, 'Cholesterol': chol.value,
            'FastingBS': fbs.value, 'MaxHR': hr.value, 'Oldpeak': oldpeak.value,
            'Sex_M': sex_m.value, 'ChestPainType_Non-anginal pain': cp_na.value,
            'RestingECG_Normal': ecg_normal.value, 'ExerciseAngina_Y': ex_angina.value,
            'ST_Slope_Up': slope_up.value,
            'ST_Slope_Flat': slope_flat.value,
            # Defaults for dummies not selected
            'ChestPainType_Asymptomatic': 0,
            'ChestPainType_Typical angina': 0,
            'RestingECG_LVH': 0 ,
            'RestingECG_ST': 0
        }

        # Ensure input matches trained features
        input_df = pd.DataFrame([input_dict])
        # Reindex with trained feature columns, filling missing values with 0
        input_df = input_df.reindex(columns=x_columns, fill_value=0)

        # Scale and predict
        input_scaled = scaler.transform(input_df)
        pred = model.predict(input_scaled)[0][0]

        # Show result
        result = "💔 High Risk" if pred > 0.5 else "💓 Low Risk"
        print(f"\n🧠 Prediction: {pred:.2f} → {result}")

    predict_btn.on_click(make_prediction)

    print("🟧 Mandatory Inputs:")
    display(age, bp, chol, fbs, hr, oldpeak)
    print("🟨 Optional Inputs:")
    display(sex_m, cp_na, ecg_normal, ex_angina, slope_up, slope_flat, predict_btn)

# 🚀 Launch App
login_screen()